# Facebook prophet model #####

The facebook prophet model is known for its capabilities in time series forecasting. The documentation can be found here: 
https://facebook.github.io/prophet/

## 1. Import packages 


In [30]:
# Install needed packages
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import utils
import numpy as np
import matplotlib.pyplot as plt
from darts.metrics import mape, mae, rmse, r2_score, mse
from prophet.serialize import model_to_json, model_from_json
from prophet.utilities import regressor_coefficients

In [31]:
# Define a function to load and prepare the dataset
def load_and_prepare_data(file_path):
    """
    Loads and prepares energy price data from the specified CSV file.
    Ensures chronological order and converts the 'Date' column to datetime format.
    
    Args:
        file_path (str): Path to the CSV file.
        
    Returns:
        pd.DataFrame: A DataFrame with the processed energy price data.
    """
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)
    df.set_index('Date', inplace=True)
    return df


## 2. Load and Prepare Data

In [32]:
# Load the full dataset
df = load_and_prepare_data('../../data/Final_data/final_data_july.csv')

# Reset the index and rename columns for Prophet compatibility
df = df.reset_index().rename(columns={'Date': 'ds', 'Day_ahead_price (€/MWh)': 'y'})

# Move the 'y' column to the second position in the DataFrame
cols = list(df.columns)
cols.remove('y')
cols.insert(1, 'y')
df = df[cols]

# Preview the data
df.head()



,ds,y,Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
0,2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.1,250.979
1,2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.0,258.671
2,2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.9,271.495
3,2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.4,270.613
4,2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.3,287.555


## 3. Load Training and Test Sets

In [33]:
# Load training and testing datasets
train_df = load_and_prepare_data('../../data/Final_data/train_df.csv')
test_df = load_and_prepare_data('../../data/Final_data/test_df.csv')

# Reset the index and rename columns for Prophet compatibility
train_df = train_df.reset_index().rename(columns={'Date': 'ds', 'Day_ahead_price (€/MWh)': 'y'})
test_df = test_df.reset_index().rename(columns={'Date': 'ds', 'Day_ahead_price (€/MWh)': 'y'})

# Reorder columns in both training and test sets
cols_train = list(train_df.columns)
cols_test = list(test_df.columns)

cols_train.remove('y')
cols_test.remove('y')

cols_train.insert(1, 'y')
cols_test.insert(1, 'y')

train_df = train_df[cols_train]
test_df = test_df[cols_test]

# Preview training data
train_df.head()

,ds,y,Solar_radiation (W/m2),Wind_speed (m/s),Temperature (°C),Biomass (GWh),Hard_coal (GWh),Hydro (GWh),Lignite (GWh),Natural_gas (GWh),Other (GWh),Pumped_storage_generation (GWh),Solar_energy (GWh),Wind_offshore (GWh),Wind_onshore (GWh),Net_total_export_import (GWh),BEV_vehicles,Oil_price (EUR),TTF_gas_price (€/MWh),Nuclear_energy (GWh)
0,2012-01-01,18.19,14.75,4.95,8.39,98.605,108.454,51.011,325.337,188.811,54.040,19.314,6.263,3.404,235.467,54.662,6,99.64,21.1,250.979
1,2012-01-02,33.82,15.12,5.00,7.41,98.605,222.656,51.862,343.168,229.293,54.166,28.892,6.312,3.350,231.772,-64.477,6,100.04,20.0,258.671
2,2012-01-03,35.03,31.88,7.77,5.23,98.605,162.204,48.851,336.773,241.297,53.518,21.072,24.226,7.292,504.484,-35.078,6,100.44,20.9,271.495
3,2012-01-04,32.16,25.21,8.04,4.78,98.605,189.633,47.101,323.976,252.289,52.194,28.300,14.157,7.828,541.528,22.924,6,103.15,21.4,270.613
4,2012-01-05,20.35,13.46,9.98,4.23,98.605,175.733,45.854,327.502,259.018,52.179,31.887,4.728,8.280,572.819,35.618,6,103.92,21.3,287.555


## 4. Instantiate the Prophet model using prophet python package

In [34]:
# Create the prophet model with the applicable seasonality and holidays
m = Prophet(
    seasonality_mode='additive',
    yearly_seasonality=25,
    weekly_seasonality=3,
    daily_seasonality=False,
    seasonality_prior_scale=1,
    holidays_prior_scale=1,
    changepoint_prior_scale=0.01,
    scaling="absmax", 
    interval_width=0.9,
) 

# Add monthly seasonality
m.add_seasonality(name='monthly', period=30.5, fourier_order=5)
m.add_seasonality(name='weekly', period=7, fourier_order=3, prior_scale=0.1)

# Add holidays for Germany to the data 
m.add_country_holidays(country_name='DE')

# Add regressors for all columns except ds and y
for column in df.columns:
    if column not in ['ds', 'y']:
        m.add_regressor(column, prior_scale=0.7, mode='additive')

# Fit the model
m.fit(train_df)


08:47:36 - cmdstanpy - INFO - Chain [1] start processing
08:47:37 - cmdstanpy - INFO - Chain [1] done processing


In [35]:
# Create the future dataframe 
future = m.make_future_dataframe(periods=test_df.shape[0], freq='1D')

# Check the alignment and number of rows
print(future['ds'].equals(df['ds']))  # Should return True if perfectly aligned
print(future.shape[0] == df.shape[0])  # Also should return True


True
True


In [36]:
# Add columns from df to future
for column in df.columns:
    if column != 'ds':  
        future[column] = df[column]

# Forecast the future
forecast = m.predict(future)
# Show results
print(forecast.tail())

             ds      trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
4588 2024-07-24  58.613751   53.409271   96.832529    55.925641    61.259509   
4589 2024-07-25  58.615656   58.005430   99.783651    55.927046    61.263888   
4590 2024-07-26  58.617561   58.366504  101.736523    55.928452    61.268267   
4591 2024-07-27  58.619466   61.303453  102.588576    55.929857    61.272646   
4592 2024-07-28  58.621371   33.880032   74.906478    55.931262    61.275764   

      Ascension Day  Ascension Day_lower  Ascension Day_upper  BEV_vehicles  \
4588            0.0                  0.0                  0.0      1.360587   
4589            0.0                  0.0                  0.0      1.360587   
4590            0.0                  0.0                  0.0      1.362373   
4591            0.0                  0.0                  0.0      1.360587   
4592            0.0                  0.0                  0.0      1.360587   

      ...     weekly  weekly_lower  weekly_u

In [37]:
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Ascension Day,Ascension Day_lower,Ascension Day_upper,BEV_vehicles,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2012-01-01,44.040443,-18.627430,22.582754,44.040443,44.040443,0.0,0.0,0.0,-0.400732,...,-13.860666,-13.860666,-13.860666,-9.337363,-9.337363,-9.337363,0.0,0.0,0.0,2.203754
1,2012-01-02,44.027629,7.460674,51.492874,44.027629,44.027629,0.0,0.0,0.0,-0.400732,...,3.478667,3.478667,3.478667,-8.488512,-8.488512,-8.488512,0.0,0.0,0.0,29.349682
2,2012-01-03,44.014815,-5.760635,38.114807,44.014815,44.014815,0.0,0.0,0.0,-0.400732,...,5.238743,5.238743,5.238743,-7.133012,-7.133012,-7.133012,0.0,0.0,0.0,16.859636
3,2012-01-04,44.002002,-3.745838,38.509366,44.002002,44.002002,0.0,0.0,0.0,-0.400732,...,4.880194,4.880194,4.880194,-5.333927,-5.333927,-5.333927,0.0,0.0,0.0,16.871298
4,2012-01-05,43.989188,-2.508093,39.468984,43.989188,43.989188,0.0,0.0,0.0,-0.400732,...,4.393367,4.393367,4.393367,-3.201393,-3.201393,-3.201393,0.0,0.0,0.0,17.685212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588,2024-07-24,58.613751,53.409271,96.832529,55.925641,61.259509,0.0,0.0,0.0,1.360587,...,4.880194,4.880194,4.880194,-0.198010,-0.198010,-0.198010,0.0,0.0,0.0,75.068397
4589,2024-07-25,58.615656,58.005430,99.783651,55.927046,61.263888,0.0,0.0,0.0,1.360587,...,4.393367,4.393367,4.393367,-0.243073,-0.243073,-0.243073,0.0,0.0,0.0,78.996728
4590,2024-07-26,58.617561,58.366504,101.736523,55.928452,61.268267,0.0,0.0,0.0,1.362373,...,2.195763,2.195763,2.195763,-0.453694,-0.453694,-0.453694,0.0,0.0,0.0,79.973838
4591,2024-07-27,58.619466,61.303453,102.588576,55.929857,61.272646,0.0,0.0,0.0,1.360587,...,-6.326068,-6.326068,-6.326068,-0.841332,-0.841332,-0.841332,0.0,0.0,0.0,82.694921


In [38]:
columns = forecast.columns
columns = list(columns)
columns

['ds',
 'trend',
 'yhat_lower',
 'yhat_upper',
 'trend_lower',
 'trend_upper',
 'Ascension Day',
 'Ascension Day_lower',
 'Ascension Day_upper',
 'BEV_vehicles',
 'BEV_vehicles_lower',
 'BEV_vehicles_upper',
 'Biomass (GWh)',
 'Biomass (GWh)_lower',
 'Biomass (GWh)_upper',
 'Christmas Day',
 'Christmas Day_lower',
 'Christmas Day_upper',
 'Easter Monday',
 'Easter Monday_lower',
 'Easter Monday_upper',
 'German Unity Day',
 'German Unity Day_lower',
 'German Unity Day_upper',
 'Good Friday',
 'Good Friday_lower',
 'Good Friday_upper',
 'Hard_coal (GWh)',
 'Hard_coal (GWh)_lower',
 'Hard_coal (GWh)_upper',
 'Hydro (GWh)',
 'Hydro (GWh)_lower',
 'Hydro (GWh)_upper',
 'Labor Day',
 'Labor Day_lower',
 'Labor Day_upper',
 'Lignite (GWh)',
 'Lignite (GWh)_lower',
 'Lignite (GWh)_upper',
 'Natural_gas (GWh)',
 'Natural_gas (GWh)_lower',
 'Natural_gas (GWh)_upper',
 'Net_total_export_import (GWh)',
 'Net_total_export_import (GWh)_lower',
 'Net_total_export_import (GWh)_upper',
 "New Year's Da

## 5. Plot the predictions vs. the actual data for the test period

In [39]:
# Define the test period start and end dates
test_start = test_df['ds'].min()
test_end = test_df['ds'].max()

# Filter forecast to test period
test_forecast = forecast[(forecast['ds'] >= test_start) & (forecast['ds'] <= test_end)]

# Filter historical data to test period
test_actuals = df[(df['ds'] >= test_start) & (df['ds'] <= test_end)]

# Create a figure with subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for forecast and actual data
fig.add_trace(
    go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat'], name='Forecast', mode='lines', line=dict(color='blue')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat_lower'], name='Lower Confidence', mode='lines', line=dict(color='gray', dash='dot')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=test_forecast['ds'], y=test_forecast['yhat_upper'], name='Upper Confidence', mode='lines', line=dict(color='gray', dash='dot'), fill='tonexty'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=test_actuals['ds'], y=test_actuals['y'], name='Actual', mode='markers', marker=dict(color='black', size=3)),
    secondary_y=False,
)

# Set graph title and axis labels
fig.update_layout(
    title='Forecast vs Actuals for Test Period',
    xaxis_title='Date',
    yaxis_title='Day-Ahead Energy Price (EUR/MWh)',
    legend=dict(x=0.01, y=0.99, bordercolor="Black", borderwidth=1)
)

# Update layout
fig.update_layout(
    title='Forecast vs Actuals for Test Period',
    xaxis_title='Date',
    yaxis_title='Day-Ahead Energy Price (EUR/MWh)',
    legend=dict(
        x=0.94,   # Set x position to 1 (far right)
        y=1,   # Set y position to 1 (top)
        xanchor='right',  # Anchor the legend's x position to the right
        yanchor='top',    # Anchor the legend's y position to the top
        bordercolor='black',  # Optional: Add a border around the legend
        borderwidth=1        # Optional: Set the border width
    ),
    template='plotly'  # Changed to plotly_white for better visibility
)

# Save the figure as a PNG image
fig.write_image("forecast_vs_actuals.png")

# Show plot
fig.show()


In [40]:
# Python
plot_components_plotly(m, forecast)

In [41]:
# Calculate the regression coefficients
coefficients = regressor_coefficients(m)
print(coefficients)

                          regressor regressor_mode      center  coef_lower  \
0            Solar_radiation (W/m2)       additive  131.208106    0.008666   
1                  Wind_speed (m/s)       additive    3.643915    0.211087   
2                  Temperature (°C)       additive    9.668547    0.567939   
3                     Biomass (GWh)       additive  118.831343    0.246731   
4                   Hard_coal (GWh)       additive  228.124833   -0.026839   
5                       Hydro (GWh)       additive   53.622936   -0.045685   
6                     Lignite (GWh)       additive  347.640043   -0.017024   
7                 Natural_gas (GWh)       additive  210.653383    0.074977   
8                       Other (GWh)       additive   61.809469    0.230972   
9   Pumped_storage_generation (GWh)       additive   20.869179   -0.075131   
10               Solar_energy (GWh)       additive  112.030649   -0.061237   
11              Wind_offshore (GWh)       additive   38.656543  

### Error metrics

In [42]:
# Ensuring that test_df and forecast are aligned by date and filter the forecast to the test period
test_forecast = forecast[(forecast['ds'] >= test_df['ds'].min()) & (forecast['ds'] <= test_df['ds'].max())]

# Making sure the lengths are the same and they are in the same order
if len(test_forecast) == len(test_df) and all(test_forecast['ds'].values == test_df['ds'].values):
    # Sklearn metrics to calculate MSE and MAE
    mse = mean_squared_error(test_df['y'], test_forecast['yhat'])
    mae = mean_absolute_error(test_df['y'], test_forecast['yhat'])
    rmse = np.sqrt(mse)  # RMSE is just the square root of MSE
    
    # MAPE
    mape = np.mean(np.abs((test_forecast['yhat'] - test_df['y']) / test_df['y'])) * 100  # Multiply by 100 to get percentage

else:
    # Raise Error
    raise ValueError("Dataframes are not aligned or of different lengths. Please check and try again.")

# Print the metrics
evaluation_metrics = {
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'MAPE': mape
}

# Convert the dictionary to a pandas dataframe for better visualization
metrics_df = pd.DataFrame([evaluation_metrics])
print(metrics_df)


           MSE        MAE       RMSE  MAPE
0  1446.587937  25.088073  38.034037   NaN


## 6. Save model for further applications


In [43]:
with open('prophet_model.json', 'w') as fout:
    fout.write(model_to_json(m))  # Save model

# with open('prophet_model.json', 'r') as fin:
   # m = model_from_json(fin.read())  # Load model
